# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Wang  ->  N. Wang  |  ['Wang']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. Lee  ->  K. Lee  |  ['K. Lee']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
T. Suhasaria  ->  T. Suhasaria  |  ['T. Suhasaria']


N. Storm  ->  N. Storm  |  ['N. Storm']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Liu  ->  J. Liu  |  ['J. Liu']
T. Henning  ->  T. Henning  |  ['T. Henning']
K. Lee  ->  K. Lee  |  ['K. Lee']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 107 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.14400


extracting tarball to tmp_2507.14400... done.
Retrieving document from  https://arxiv.org/e-print/2507.14707


extracting tarball to tmp_2507.14707...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.14708


...

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.15790/ms-upload.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'ref2' from 'tmp_2507.15790/ref2.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'ref1' from 'tmp_2507.15790/ref1.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.15819/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'ShortPessimisticOnlyIsocurvatureTable1' from 'tmp_2507.15819/ShortPessimisticOnlyIsocurvatureTable1.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 220 bibliographic references in tmp_2507.15819/aa.bbl.
Issues with the citations
syntax error in line 5: unbalanced braces


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Ach{ú}carro} {et~al.}(2022){Ach{ú}carro}, {Biagetti}, {Braglia}, {Cabass}, {Castorina}, {Caldwell}, {Chen}, {Coulton}, {Flauger}, {Fumagalli}, {Ivanov}, {Lee}, {Maleknejad}, {Meerburg}, {Moradinezhad Dizgah}, {Palma}, {Renaux-Petel}, {Pimentel}, {Wallisch}, {Wandelt}, {Witkowski},  {Kimmy Wu}}]{Achucarro_2022}{Ach{ú}carro}, A., {Biagetti}, M., {Braglia}, M., {et~al.} \href{https://ui.adsabs.harvard.edu/abs/2022arXiv220308128A}{2022, arXiv:2203.08128}
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3}).*?href(.*?{(?P<url>http[\S]*)})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Alvarez} {et~al.}(2014){Alvare

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.14905-b31b1b.svg)](https://arxiv.org/abs/2507.14905) | **A deep Search for Ethylene Glycol and Glycolonitrile in the V883 Ori Protoplanetary Disk**  |
|| A. M. A. Fadul, et al. -- incl., <mark>T. Suhasaria</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *15 pages, 8 figures, accepted for publication in ApJL*|
|**Abstract**|            Ethylene glycol ($\mathrm{(CH_2OH)_2}$, hereafter EG) and Glycolonitrile ($\mathrm{HOCH_2CN}$, hereafter GN) are considered molecular precursors of nucleic acids. EG is a sugar alcohol and the reduced form of Glycolaldehyde ($\mathrm{CH_2(OH)CHO}$, hereafter GA). GN is considered a key precursor of adenine formation (nucleotide) and can be a precursor of glycine (amino acid). Detections of such prebiotic molecules in the interstellar medium are increasingly common. How much of this complexity endures to the planet formation stage, and thus is already present when planets form, remains largely unknown. Here we report Atacama Large Millimeter/sub-millimeter Array (ALMA) observations in which we tentatively detect EG and GN in the protoplanetary disk around the outbursting protostar V883 Ori. The observed EG emission is best reproduced by a column density of $\mathrm{3.63^{+0.11}_{-0.12} \times 10^{16} \; cm^{-2}}$ and a temperature of at least 300 K. The observed GN emission is best reproduced by a column density of $\mathrm{3.37^{+0.09}_{-0.09} \times 10^{16} \; cm^{-2}}$ and a temperature of $88^{+1.2}_{-1.2}$ K. Comparing the abundance of EG and GN relative to methanol in V883 Ori with other objects, V883 Ori falls between hot cores and comets in terms of increasing complexity. This suggests that the build up of prebiotic molecules continues past the hot core phase into the epoch of planet formation. Nascent planets in such environments may inherit essential building blocks for life, enhancing their potential habitability. Further observations of this protoplanetary disk at higher spectral resolution are required to resolve blended lines and to confirm these tentative detection.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15122-b31b1b.svg)](https://arxiv.org/abs/2507.15122) | **Stellar Population Astrophysics (SPA) with the TNG: NLTE atmospheric parameters and abundances of giant stars in 33 Open Clusters**  |
|| M. D. Ponte, et al. -- incl., <mark>N. Storm</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *23 pages, 12 figures, Accepted for publication in A&A*|
|**Abstract**|            Open clusters serve as important tools for accurately studying the chemical evolution of the Milky Way. By combining precise chemical data from high-resolution spectra with information on their distances and ages, we can effectively uncover the processes that have shaped our Galaxy. This study aims to derive NLTE atmospheric parameters and chemical abundances for approximately one hundred giant stars across 33 open clusters with near-solar metallicity. The clusters span a wide range of ages, enabling an assessment of the presence and extent of any age-related abundance gradients. In the Stellar Population Astrophysics (SPA) project, we acquired new high-resolution spectra of open clusters using the HARPS-N echelle spectrograph at the Telescopio Nazionale Galileo. We chemically characterized nine open clusters for the first time and reanalyzed previously studied SPA clusters, resulting in a consistent and homogeneous sample. We determined NLTE atmospheric parameters using the equivalent width method and derived NLTE chemical abundances through spectral synthesis for various elements, including alpha elements (Mg, Si, and Ti), light odd-Z elements (Na, Al), iron-peak elements (Mn, Co, and Ni), and neutron-capture elements (Sr, Y, and Eu). Our findings are compared with the existing literature, revealing good agreement. We examine the trends of [X/Fe] versus age, confirming previous observations and the enrichment patterns predicted by nucleosynthesis processes. Positive correlations with age are present for Mg, Si, Ti, Al, Mn, Co, Ni, and Sr, while Na and Y and Eu show a negative trend. This study emphasizes the significance of NLTE corrections and reinforces the utility of open clusters as tracers of Galactic chemical evolution. Furthermore, we provide a benchmark sample of NLTE abundances for upcoming open cluster surveys within large-scale projects such as 4MOST and WEAVE.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15516-b31b1b.svg)](https://arxiv.org/abs/2507.15516) | **The CARMENES search for exoplanets around M dwarfs. Revisiting the GJ 317, GJ 463, and GJ 3512 systems and two newly discovered planets orbiting GJ 9773 and GJ 508.2**  |
|| J. C. Morales, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *32 pages (including appendix with radial velocity time series), 16 figures, 14 tables. Accepted for publication in A&A*|
|**Abstract**|            Surveys for exoplanets indicate that the occurrence rate of gas giant planets orbiting late-type stars in orbits with periods shorter than 1000 days is lower than in the case of Sun-like stars. This is in agreement with planet formation models based on the core or pebble accretion paradigm. The CARMENES exoplanet survey has been conducting radial-velocity observations of several targets that show long-period trends or modulations that are consistent with the presence of giant planets at large orbital separations. We present an analysis of five such systems that were monitored with the CARMENES spectrograph, as well as with the IRD spectrograph. In addition, we used archival data to improve the orbital parameters of the planetary systems. We improve the parameters of three previously known planets orbiting the M dwarfs GJ 317, GJ 463, and GJ 3512. We also determine the orbital parameters and minimum mass of the planet GJ 3512 c, for which only lower limits had been given previously. Furthermore, we present the discovery of two new giant planets orbiting the stars GJ 9733 and GJ 508.2, although for the second one only lower limits to the orbital properties can be determined. The new planet discoveries add to the short list of known giant planets orbiting M-dwarf stars with subsolar metallicity at long orbital periods above 2000 days. These results reveal that giant planets appear to form more frequently in wide orbits than in close-in orbits around low-mass and lower metallicity stars.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15819-b31b1b.svg)](https://arxiv.org/abs/2507.15819) | **Euclid preparation: Expected constraints on initial conditions**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *Abstract abridged, 25 pages, 6 tables, 11 figures*|
|**Abstract**|            The Euclid mission of the European Space Agency will deliver galaxy and cosmic shear surveys, which will be used to constrain initial conditions and statistics of primordial fluctuations. We present highlights for the Euclid scientific capability to test initial conditions beyond LCDM with the main probes, i.e. 3D galaxy clustering from the spectroscopic survey, the tomographic approach to 3x2pt statistics from photometric galaxy survey, and their combination. We provide Fisher forecasts from the combination of Euclid spectroscopic and photometric surveys for spatial curvature, running of the spectral index of the power spectrum of curvature perturbations, isocurvature perturbations, and primordial features. For the parameters of these models we also provide the combination of Euclid forecasts (pessimistic and optimistic) with current and future measurements of the cosmic microwave background (CMB) anisotropies., i.e. Planck, the Simons Observatory (SO), and CMB-S4. We provide Fisher forecasts for how the power spectrum and bispectrum from the Euclid spectroscopic survey will constrain the local, equilateral, and orthogonal shapes of primordial non-Gaussianity. We also review how Bayesian field-level inference of primordial non-Gaussianity can constrain local primordial non-Gaussianity. We show how Euclid, with its unique combination of the main probes, will provide the tightest constraints on low redshift to date. By targeting a markedly different range in redshift and scale, Euclid's expected uncertainties are complementary to those obtained by CMB primary anisotropy, returning the tightest combined constraints on the physics of the early Universe.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.14707-b31b1b.svg)](https://arxiv.org/abs/2507.14707) | **A prolific repeating fast radio burst source and a crisis of the magnetar model**  |
|| J.-S. Zhang, et al. -- incl., <mark>J. Li</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *33 pages, 6 figures, the 1st of papers from the FAST FRB Key Science Project Collaboration on FRB 20240114A: Energy Budget Analysis*|
|**Abstract**|            Magnetars are widely considered to be the central engine of fast radio bursts (FRBs), capable of driving coherent radio emission through magnetic energy release. The accumulation of burst energy can be employed as a probe to reveal the nature of FRB energy engine. Current observations of repeaters have not yet provided sufficient constraints on the magnetic energy budget for magnetar models. Here we report 11,553 bursts detected from the hyper-active repeating source FRB 20240114A over 214 days during a recent active episode, with a peak burst rate as high as 729 hr$^{-1}$, above the detection threshold of the monitoring observations using Five-hundred-meter Aperture Spherical radio Telescope (FAST). This represents the largest recorded burst sample from a single FRB source, comprising more than four times as many bursts as any previously known repeater and exceeding the cumulative total of all published bursts to date. Assuming a typical value of radio efficiency of $10^{-4}$ and a beaming factor of 0.1, the estimated total isotropic burst energy of this source exceeds 86\% of the dipolar magnetic energy of a typical magnetar with a polar surface magnetic field of $10^{15}$ G and a radius of $10^6$ cm. The total energy release of this source is about one and a half orders of magnitude higher than that of other known repeaters. We further provide the most stringent lower limit of $\bf{4.7\times10^{32}}$ G cm$^3$ on the magnetic moment of the underlying magnetar. The findings suggest that either the central engine emits with exceptionally high radio efficiency, possesses a small beaming factor, or it should be a more powerful compact object than typically considered in current magnetar models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.14708-b31b1b.svg)](https://arxiv.org/abs/2507.14708) | **Searching for periodic signals and quasi-periodic oscillations from an extremely active cycle of FRB 20240114A**  |
|| D. Zhou, et al. -- incl., <mark>K. Lee</mark>, <mark>J. Li</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *23 pages, 10 figures, 2 tables, the 2nd of papers from the FAST FRB Key Science Project Collaboration on FRB 20240114A: (quasi-) period search*|
|**Abstract**|            Possible periodic features in fast radio bursts (FRBs) may provide insights into their astrophysical origins. Using extensive observations from the Five-hundred-meter Aperture Spherical radio Telescope (FAST), we conduct a search for periodicity across multiple timescales in the exceptionally active repeater source FRB 20240114A. A detailed analysis of 11,553 bursts reveals three candidate periodic signals (3.4-6$\sigma$), with frequencies of approximately 1.486 Hz, 1.576 Hz, and 1.865 Hz. This finding offers tentative evidence for a potential association with magnetar spin, although further studies are necessary for definitive confirmation. On longer timescales, a 67.5-69.2 day activity cycle is identified using two independent methods, with a significance exceeding $3.5\sigma$. In the burst time series, quasi-periodic oscillations (QPOs) in the range of a few hundred Hz are identified, with a significance exceeding 5$\sigma$, and a strong correlation is found between their central frequencies and widths (Pearson $R=0.97$). The decomposed temporal profiles of these QPOs suggest an amplitude-modulated quasi-periodic signal. The burst clusters manifest periodic burst trains ranging from a few milliseconds to several tens of milliseconds, revealing transient periodic characteristics. Despite the unprecedentedly large sample size from a single repeating FRB source during a time-continuous active period, no stable periodic signals are detected. Instead, there are multiple QPOs that dynamically evolve over time. These results clarify the bias of previous studies that occasionally detected QPOs believed to be potentially related to the spin of compact objects.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.14711-b31b1b.svg)](https://arxiv.org/abs/2507.14711) | **Investigating FRB 20240114A with FAST: Morphological Classification and Drifting Rate Measurements in a Burst-Cluster Framework**  |
|| L.-X. Zhang, et al. -- incl., <mark>J. Li</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *22 pages, 13 figures, 1 tables, the 3rd of papers from the FAST FRB Key Science Project Collaboration on FRB 20240114A: Burst Morphology Analysis*|
|**Abstract**|            This study investigates the morphological classification and drifting rate measurement of the repeating fast radio burst (FRB) source FRB20240114A using the Five-hundred-meter Aperture Spherical Telescope (FAST). Detected on January 14, 2024, FRB20240114A showed an exceptionally high burst rate. During a continuous 15,780-second monitoring session on March 12, 2024, 3,203 bursts (2,109 burst-clusters) were detected. We refine the definitions of sub-bursts, bursts, and burst-clusters. Using an average dispersion measure (DM) of 529.2 pc cm$^{-3}$, burst-clusters were classified into Downward Drifting, Upward Drifting, No Drifting, No Evidence for Drifting, Not-Clear, and Complex categories. Notably, 233 (23.82%) of the 978 drifting burst-clusters showed upward drifting. Excluding 142 upward drifting single-component clusters, the remaining 91 upward drifting double- or multiple-components clusters account for 10.89% of the drifting sample. Further restricting to those with consecutive time intervals, only 9 upward drifting bursts remain. An inverse correlation is found between drifting rate and sub-burst effective width. Upward drifting single-component clusters tend to have smaller effective widths, bandwidths, and fluxes than downward drifting ones. For these upward drifting clusters, drifting rate increases with peak frequency. A Kolmogorov-Smirnov test reveals longer consecutive intervals in upward drifting clusters compared to downward ones, suggesting differing underlying mechanisms.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15482-b31b1b.svg)](https://arxiv.org/abs/2507.15482) | **Delayed Launch of Ultrafast Outflows in the Tidal Disruption Event AT2020afhd**  |
|| Z. Lin, et al. -- incl., <mark>Y. Wang</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *15 pages, 7 figures, 4 tables; accepted for publication in ApJ Letters*|
|**Abstract**|            We report the detection and characterization of ultrafast outflows (UFOs) in the X-ray spectra of the tidal disruption event (TDE) AT2020afhd, based on observations from NICER, Swift, and XMM. Prominent blueshifted absorption features were detected exclusively during the intermediate phase of the event, occurring between days 172 and 212 within the first 300 days post-discovery. During this period, the UFO appeared no earlier than day 74, strengthened between days 172 and 194, and disappeared after day 215. This marks the first time that the full evolutionary sequence of X-ray outflows has been observed in a TDE. Moreover, the outflows exhibited a dramatic deceleration from ~0.19c to ~0.0097c over a span of approximately 10 days. Photoionization spectral analysis reveals an inverse correlation between outflow velocity and ionization parameter, in contradiction to the predictions from radiation pressure-driven wind. Eventually, we propose that the delayed onset of the outflows may result from an increase in the wind opening angle and/or metal enrichment, particularly iron and oxygen, during the disk formation phase.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15790-b31b1b.svg)](https://arxiv.org/abs/2507.15790) | **Decadal evolution of a repeating fast radio burst source**  |
|| P. Wang, et al. -- incl., <mark>K. Lee</mark>, <mark>J. Li</mark>, <mark>Y. Wang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-22*|
|*Comments*| *85 pages, 14 figures, under review of Nature Astronomy*|
|**Abstract**|            The origin of fast radio bursts (FRBs), the brightest cosmic radio explosions, is still unknown. Bearing critical clues to FRBs' origin, the long-term evolution of FRBs has yet to be confirmed, since the field is still young and most FRBs were seen only once. Here we report clear evidence of decadal evolution of FRB~20121102A, the first precisely localized repeater. In conjunction with archival data, our FAST and GBT monitoring campaign since 2020 reveals a significant 7% decline of local dispersion measure (DM). The rotation measure (RM) of 30,755$\pm$16 $\mathrm{rad\,m^{-2}}$ detected in the last epoch represents a 70% decrease compared to that from December 2016. The $\sigma_{RM}$ parameter, which describes the complexity of the magneto-ionic environment surrounding the source, was shown to have decreased by 13%. These general trends reveal an evolving FRB environment, which could originate from an early-phase supernova associated with an enhanced pair wind from the FRB central engine.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.14905/./Figures/prebiotic_spectra.png', 'tmp_2507.14905/./Figures/summed_spectra.png', 'tmp_2507.14905/./Figures/leedata1.png']
copying  tmp_2507.14905/./Figures/prebiotic_spectra.png to _build/html/
copying  tmp_2507.14905/./Figures/summed_spectra.png to _build/html/
copying  tmp_2507.14905/./Figures/leedata1.png to _build/html/
exported in  _build/html/2507.14905.md
    + _build/html/tmp_2507.14905/./Figures/prebiotic_spectra.png
    + _build/html/tmp_2507.14905/./Figures/summed_spectra.png
    + _build/html/tmp_2507.14905/./Figures/leedata1.png
found figures ['tmp_2507.15122/./figs/elements_fe_age_galah_errors.png', 'tmp_2507.15122/./figs/elements_age_fe_linear_regression_OLS.png', 'tmp_2507.15122/./figs/lotus_results_5_clusters_final_nlte_lte.png']
copying  tmp_2507.15122/./figs/elements_fe_age_galah_errors.png to _build/html/
copying  tmp_2507.15122/./figs/elements_age_fe_linear_regression_OLS.png to _build/html/
copying  tmp_2507.15122/./figs/lotus_result

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A deep Search for Ethylene Glycol and Glycolonitrile in the V883 Ori Protoplanetary Disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.14905-b31b1b.svg)](https://arxiv.org/abs/2507.14905)<mark>Appeared on: 2025-07-22</mark> -  _15 pages, 8 figures, accepted for publication in ApJL_

</div>
<div id="authors">

A. M. A. Fadul, et al. -- incl., <mark>T. Suhasaria</mark>

</div>
<div id="abstract">

**Abstract:** Ethylene glycol ( $\mathrm{(CH_2OH)_2}$ , hereafter EG) and Glycolonitrile ( $\mathrm{HOCH_2CN}$ , hereafter GN) are considered molecular precursors of nucleic acids. EG is a sugar alcohol and the reduced form of Glycolaldehyde ( $\mathrm{CH_2(OH)CHO}$ , hereafter GA). GN is considered a key precursor of adenine formation (nucleotide) and can be a precursor of glycine (amino acid).Detections of such prebiotic molecules in the interstellar medium are increasingly common. How much of this complexity endures to the planet formation stage, and thus is already present when planets form, remains largely unknown. Here we report Atacama Large Millimeter/sub-millimeter Array (ALMA) observations in which we tentatively detect EG and GN in the protoplanetary disk around the outbursting protostar V883 Ori. The observed EG emission is best reproduced by a column density of $\mathrm{3.63^{+0.11}_{-0.12} \times 10^{16} \; cm^{-2}}$ and a temperature of at least 300 K. The observed GN emission is best reproduced by a column density of $\mathrm{3.37^{+0.09}_{-0.09} \times 10^{16} \; cm^{-2}}$ and a temperature of $88^{+1.2}_{-1.2}$ K.Comparing the abundance of EG and GN relative to methanol in V883 Ori with other objects, V883 Ori falls between hot cores and comets in terms of increasing complexity. This suggests that the build up of prebiotic molecules continues past the hot core phase into the epoch of planet formation. Nascent planets in such environments may inherit essential building blocks for life, enhancing their potential habitability. Further observations of this protoplanetary disk at higher spectral resolution are required to resolve blended lines and to confirm these tentative detection.

</div>

<div id="div_fig1">

<img src="tmp_2507.14905/./Figures/prebiotic_spectra.png" alt="Fig5" width="100%"/>

**Figure 5. -** The best-fit model of the detected transitions of $\mathrm{g\text{'}Gg-(CH_2OH)_2}$ is shown in green, and $\mathrm{HOCH_2CN}$ is shown in orange. These are plotted on top of the data, which is shown in black, along with the summed spectra of the other detected molecules by Fadul25, shown in blue. The dotted and dashed gray line indicates the noise level in the data (corresponding to 1 and 3$\sigma$). (*results*)

</div>
<div id="div_fig2">

<img src="tmp_2507.14905/./Figures/summed_spectra.png" alt="Fig2" width="100%"/>

**Figure 2. -** Represent the cumulative model for all species reported by Fadul25 in blue, and the same cumulative model include EG and GN in green. The black represents the observed spectra, and the gray dotted horizontal line represents the noise level, which corresponds to 1$\sigma$.  (*summed_spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2507.14905/./Figures/leedata1.png" alt="Fig3" width="100%"/>

**Figure 3. -** Shows ethylene glycol spectra in steel blue overlaid on the observed spectra from ALMA Band 6 in black   ([Jeong, Lee and Lee 2025]()) . (*firstFigure*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.14905"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Stellar Population Astrophysics (SPA) with the TNG$\thanks{Based on observations made with the Italian TelescopioNazionale Galileo (TNG) operated on the island of La Palma bythe Fundación Galileo Galilei of the INAF (Istituto Nazionaledi Astrofisica) at the Observatorio del Roque de los Muchachos.}$: NLTE atmospheric parameters and abundances of giant stars in 33 Open Clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.15122-b31b1b.svg)](https://arxiv.org/abs/2507.15122)<mark>Appeared on: 2025-07-22</mark> -  _23 pages, 12 figures, Accepted for publication in A&A_

</div>
<div id="authors">

M. D. Ponte, et al. -- incl., <mark>N. Storm</mark>

</div>
<div id="abstract">

**Abstract:** Open clusters serve as important tools for accurately studying the chemical evolution of the Milky Way. By combining precise chemical data from high-resolution spectra with information on their distances and ages, we can effectively uncover the processes that have shaped our Galaxy. This study aims to derive non-local thermodynamic equilibrium (NLTE) atmospheric parameters and chemical abundances for approximately one hundred giant stars across 33 open clusters with near-solar metallicity. The clusters span a wide range of ages, enabling an assessment of the presence and extent of any age-related abundance gradients. In the Stellar Population Astrophysics (SPA) project, we acquired new high-resolution spectra of giant stars in a sample of open clusters using the HARPS-N echelle spectrograph at the Telescopio Nazionale Galileo (TNG). We chemically characterized nine open clusters for the first time and reanalyzed previously studied SPA clusters, resulting in a consistent and homogeneous sample. We determined NLTE atmospheric parameters using the equivalent width method and derived NLTE chemical abundances through spectral synthesis for various elements, including $\alpha$ elements (Mg, Si, and Ti), light odd-Z elements (Na, Al), iron-peak elements (Mn, Co, and Ni), and neutron-capture elements (Sr, Y, and Eu). Our findings are compared with the existing literature, revealing good agreement. We examine the trends of [ X/Fe ] versus age, confirming previous observations and the enrichment patterns predicted by nucleosynthesis processes. Positive correlations with age are present for $\alpha$ elements such as Mg, Si, Ti, odd-Z Al, and iron-peak elements Mn, Co, Ni, ${and Sr}$ , while Na and ${neutron-capture Y and Eu}$ show a negative trend. This study emphasizes the significance of NLTE corrections and reinforces the utility of open clusters as tracers of Galactic chemical evolution. Furthermore, we provide a benchmark sample of NLTE abundances for upcoming open cluster surveys within large-scale projects such as 4MOST and WEAVE.

</div>

<div id="div_fig1">

<img src="tmp_2507.15122/./figs/elements_fe_age_galah_errors.png" alt="Fig9" width="100%"/>

**Figure 9. -** Abundance [X/Fe] ratios as function of [Fe/H]. The gray diamonds represent the OCs compilation from GALAH (see Table \ref{tab:galah_oc}). Our results are color-coded according to the age. (*fig:elem_fe*)

</div>
<div id="div_fig2">

<img src="tmp_2507.15122/./figs/elements_age_fe_linear_regression_OLS.png" alt="Fig10" width="100%"/>

**Figure 10. -** Abundance [X/Fe] ratios as a function of age. The gray diamonds represent the OCs compilation from GALAH (see Table \ref{tab:galah_oc}) and the gray squares represent the OCs from OCCAM. Our results are color-coded according to [Fe/H]. (*fig:elem_age_fe*)

</div>
<div id="div_fig3">

<img src="tmp_2507.15122/./figs/lotus_results_5_clusters_final_nlte_lte.png" alt="Fig8" width="100%"/>

**Figure 8. -** Atmospheric parameters [Fe/H], $T_{\rm eff}$, and $\log g$ obtained with {\tt LOTUS} for clusters containing more than five stars. The first row presents the NLTE results, while the second row shows the LTE results. (*fig:lotus*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.15122"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\ms}{m s^{-1}}$
$\newcommand{\mjup}{M_{\rm Jup}}$
$\newcommand{\lnZ}{\ln \mathcal{Z}}$
$\newcommand{\lnL}{\ln \mathcal{L}}$
$\newcommand{\aunc}[2]{^{+#1}_{#2}}$
$\newcommand{\Comm}[1]{\textcolor{red}{\textbf{#1}}}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs: Revisiting the GJ 317, GJ 463, and GJ 3512 systems and two newly discovered planets orbiting GJ 9773 and GJ 508.2

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.15516-b31b1b.svg)](https://arxiv.org/abs/2507.15516)<mark>Appeared on: 2025-07-22</mark> -  _32 pages (including appendix with radial velocity time series), 16 figures, 14 tables. Accepted for publication in A&A_

</div>
<div id="authors">

J. C. Morales, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Surveys for exoplanets indicate that the occurrence rate of gas giant planets orbiting late-type stars in orbits with periods shorter than 1000 days is lower than in the case of Sun-like stars. This is in agreement with planet formation models based on the core or pebble accretion paradigm. The CARMENES exoplanet survey has been conducting radial-velocity observations of several targets that show long-period trends or modulations that are consistent with the presence of giant planets at large orbital separations. We present an analysis of five such systems that were monitored with the CARMENES spectrograph, as well as with the IRD spectrograph. In addition, we used archival data to improve the orbital parameters of the planetary systems. We improve the parameters of three previously known planets orbiting the M dwarfs GJ 317, GJ 463, and GJ 3512. We also determine the orbital parameters and minimum mass of the planet GJ 3512 c, for which only lower limits had been given previously. Furthermore, we present the discovery of two new giant planets orbiting the stars GJ 9733 and GJ 508.2, although for the second one only lower limits to the orbital properties can be determined. The new planet discoveries add to the short list of known giant planets orbiting M-dwarf stars with subsolar metallicity at long orbital periods above 2000 days. These results reveal that giant planets appear to form more frequently in wide orbits than in close-in orbits around low-mass and lower metallicity stars.

</div>

<div id="div_fig1">

<img src="tmp_2507.15516/./Fig5.png" alt="Fig2" width="100%"/>

**Figure 2. -** Periodogram of the residuals from the fits to the RV datasets of GJ 3512, GJ 317, GJ 463, and GJ 9773 (from top to bottom). In each panel the solid black line corresponds to the joint periodogram of all datasets, while solid lines depict the periodogram for each individual dataset as labelled. The top panel for each target illustrates the window function. Solid horizontal grey lines show the 0.1\%(dashed), 1\%(dot-dashed), and 10\%(dotted) FAP levels. (*fig:res_periodogram*)

</div>
<div id="div_fig2">

<img src="tmp_2507.15516/./Fig1a.png" alt="Fig1.1" width="50%"/><img src="tmp_2507.15516/./Fig1b.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Clustering periodogram analysis \citep{Kemmer2025} for the activity indices of GJ 3512 (top) and GJ 9773 (bottom). The 0.1\% FAP threshold is shown as a horizontal dotted grey line, while vertical dashed lines mark the position of common periods caused by the sampling of data and their harmonics as labelled. Symbols with different colours correspond to clusters of periods that appear on different activity indicators. The mean values are reported in the legend. (*fig:clustering*)

</div>
<div id="div_fig3">

<img src="tmp_2507.15516/./Fig9.png" alt="Fig3" width="100%"/>

**Figure 3. -** Parameters of the fits to the RV time series of GJ 508.2 as a function of the orbital period. The differential $\lnL$  of the fit with respect to the maximum value, the fitted RV semi-amplitude and eccentricity, and the corresponding planet minimum mass are shown from top to bottom. Dahsed blue and solid black lines correspond, respectively, to fits considering only RVs from CARMENES VIS, and considering both VIS and NIR RVs simultaneously. The dot-dashed blue line in the top panel indicates the threshold corresponding to $\Delta$\lnL=5, and the vertical dotted lines, the corresponding orbital period. (*fig:pfix_gj508*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.15516"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bk}{\bm{k}}$
$\newcommand{\bq}{\bm{q}}$
$\newcommand{\bx}{\bm{x}}$
$\newcommand{\fnl}{f_{\rm NL}}$
$\newcommand{\fnll}{f_{\mathrm{NL}}^{\mathrm{loc}}}$
$\newcommand{\fnle}{f_{\mathrm{NL}}^{\mathrm{equil}}}$
$\newcommand{\fnloC}{f_{\mathrm{NL}}^{\mathrm{ortho-CMB}}}$
$\newcommand{\fnloL}{f_{\mathrm{NL}}^{\mathrm{ortho-LSS}}}$
$\newcommand{\Mpc}{\ensuremath{\text{h/Mpc}}\xspace}$
$\newcommand{\GCsp}{{\text{GC}\ensuremath{_\mathrm{sp}}}}$
$\newcommand{\GCph}{{\text{GC}\ensuremath{_\mathrm{ph}}}}$
$\newcommand{\Omegam}{\ensuremath{\Omega_{\mathrm{m},0}}}$
$\newcommand{\Omegab}{\ensuremath{\Omega_{\mathrm{b},0}}}$
$\newcommand{\OmegaDE}{\ensuremath{\Omega_{\mathrm{DE},0}}}$
$\newcommand{\ns}{n_{\mathrm{s}}}$
$\newcommand{\lcdm}{\ensuremath{\Lambda\mathrm{CDM}}}$
$\newcommand{\logfr}{\ensuremath{\log_{10}(|f_{R0}|)}}$
$\newcommand{\fr}{\ensuremath{|f_{R0}|}}$
$\newcommand{\sg}{\ensuremath{\sigma_{8}}}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\bpd}{{b_{\phi ,   \delta}}}$
$\newcommand{\borg}{\texttt{BORG}\xspace}$
$\newcommand{\Mpch}{\ensuremath{h^{-1} \text{Mpc}}}$
$\newcommand{\dd}{\mathrm{d}}$
$\newcommand{\jvc}[1]{\textcolor{red}{JV: #1}}$
$\newcommand{\aanote}[1]{\textcolor{purple}{AA: #1}}$
$\newcommand{\DK}[1]{\textcolor{orange}{DK: #1}}$
$\newcommand{\FF}[1]{\textcolor{green}{FF: #1}}$
$\newcommand{\mb}[1]{{\color{blue} #1}}$
$\newcommand{\MB}[1]{\textcolor{orange}{{\bf[MB: #1]}}}$
$\newcommand{\YA}[1]{\textcolor{red}{{\bf[YA: #1]}}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\vspace*{-0.3cm}$Euclid preparation. Expected constraints on initial conditions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.15819-b31b1b.svg)](https://arxiv.org/abs/2507.15819)<mark>Appeared on: 2025-07-22</mark> -  _Abstract abridged, 25 pages, 6 tables, 11 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $* Euclid*$ mission of the European Space Agency will deliver galaxy and cosmic shear surveys, which will be used to constrain initial conditions and statistics of primordial fluctuations. We present highlights for the $* Euclid*$ scientific capability to test initial conditions beyond $\Lambda$ CDMwith the 3-dimensional galaxy clustering from the spectroscopic survey, the tomographic approach to $3\times2$ pt statistics (information) from photometric galaxy survey, and their combination. We then present how these $* Euclid*$ results can be enhanced when combined with current and future measurements of the cosmic microwave background (CMB) anisotropies. We provide Fisher forecasts from the combination of $* Euclid*$ spectroscopic and photometric surveysfor spatial curvature, running of the spectral index of the power spectrum of curvature perturbations, isocurvature perturbations, and primordial features. For the parameters of these models we also provide the combination of $* Euclid*$ forecasts (pessimistic and optimistic) with three different CMB specifications, i.e. $* Planck*$ , the Simons Observatory (SO), and CMB-S4. We provide Fisher forecasts for how the power spectrum and bispectrum from the $* Euclid*$ spectroscopic survey will constrain the local, equilateral, and orthogonal shapes of primordial non-Gaussianity. We also review how Bayesian field-level inference of primordial non-Gaussianity can constrain local primordial non-Gaussianity. We find that the combination of the $* Euclid*$ main probes will provide the uncertainty of $\sigma(\Omega_K) = 0.0044 (0.003)$ at the $68 \%$ confidence level (CL) in the pessimistic (optimistic) settings assuming flat spatial sections as fiducial cosmology. We also find that the combination of the $* Euclid*$ main probes can detect the running of the scalar spectral index for the fiducial value $\alpha_s = - 0.01$ with approximately $2\sigma$ ( $4\sigma$ ) uncertainty and provide the uncertainty of $\sigma(\alpha_s)  = 0.004 (0.0015)$ for the fiducial value $\alpha_s = - 0.001$ at the $68\%$ CL, always with pessimistic (optimistic) settings. We show how $* Euclid*$ will have the capability to provide constraints on isocurvature perturbations with ablue spectral index that are one order of magnitude tighter than current bounds. The combined power spectrum and bispectrum Fisher forecast for the $* Euclid*$ spectroscopic survey leads to $\sigma (f_{\rm NL}^{\rm local}) = 2.2$ , $\sigma (f_{\rm NL}^{\rm equil}) = 108$ , and $\sigma (\fnl^{\rm ortho}) = 33$ by assuming $k_{\rm max} = 0.15 h {\rm Mpc}^{-1}$ and universality for the halo mass function. We show how the expected uncertainty on the local shape decreases by encompassing summary statistics and using Bayesian field-level inference. For the $* Euclid*$ main probes, we find relative errors on the amplitude of primordial oscillations (with a fiducial value of $0.01$ ), of 21 \% (18 \% ) for linear frequency and of 22 \% (18 \% ) for logarithmic frequency at $68.3\%$ CL in the pessimistic (optimistic) case. These uncertainties can be further improved by adding the information from the bispectrum and the non-linear reconstruction. We show how $* Euclid*$ , with its unique combination of 3-dimensional galaxy clustering from the spectroscopic survey and $3\times2$ pt statistics from the photometric survey, will provide the tightest constraints on low redshift to date. By targeting a markedly different range in redshift and scale, $* Euclid*$ 's expected uncertainties are complementary to those obtained by CMB primary anisotropy, returning the tightest combined constraints on the physics of the early Universe.

</div>

<div id="div_fig1">

<img src="tmp_2507.15819/./JV_EuclidIsocurvature1aTickIn.png" alt="Fig10.1" width="50%"/><img src="tmp_2507.15819/./JV_EuclidIsocurvature1bTickIn.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** 
Matter power spectra at redshift $z=0$.
The left panel shows the linear matter power spectrum today, if the primordial perturbations are scale-invariant pure adiabatic perturbations with amplitude $\mathcal{P}_\mathcal{RR} = 25\times10^{-10}$(red dotted-dashed), or if the primordial perturbations are scale-invariant pure cold dark matter isocurvature perturbations with amplitude $\mathcal{P}_\mathcal{II} = 25\times10^{-10}$(blue dashed), and the resulting correlation component to the matter power spectrum if there are these two components and they are $+100\%$ correlated with each other (green solid).
The right panel shows the fiducial model used in the case of "maximum $2\sigma$ allowed by * Planck*." The linear total matter power spectrum (tot lin) is the sum of the adiabatic (adi) and blue-tilted isocurvature (iso) contributions, and their $\sim10\%$ correlation (cor). This (tot lin) is then corrected by \texttt{Halofit} to approximate the non-linear spectrum (non-lin) of the fiducial model. (*fig:isocurvatureexamples*)

</div>
<div id="div_fig2">

<img src="tmp_2507.15819/./JV_PII2_kmax_TickIn.png" alt="Fig2" width="100%"/>

**Figure 2. -** Fisher-matrix marginalised uncertainty for $10^{10}$ times the primordial isocurvature perturbation amplitude $\mathcal{P}_\mathcal{II}$ at $k_2 = 0.1 $Mpc$^{-1}$ for Case 1 "maximum $2\sigma$ allowed by * Planck*" (solid lines), Case 2 "scale-invariant uncorrelated $n_\mathrm{iso}=1.0$" (dashed lines), and Case 3 "blue-tilted uncorrelated $n_\mathrm{iso}=2.9$" (dotted lines).
Our baseline results of Table \ref{tab:isocurvaturefisher} are for the pessimistic setting. To test the effect of the choice of $k_{\rm max}$ and $\ell_{\rm max}$ on the results we repeat the analysis with an even more conservative quasilinear setting ($k_{\rm max} = 0.15 h {\rm Mpc}^{-1}$ for $\GCsp$, $\ell_{\rm max} = 750$ for WL, and $\ell_{\rm max} = 500$ for $\GCph$ and XC) and with the optimistic setting ($k_{\rm max} = 0.30 h {\rm Mpc}^{-1}$ for $\GCsp$, $\ell_{\rm max} = 5000$ for WL, and $\ell_{\rm max} = 3000$ for $\GCph$ and XC). (*fig:isoc_quasi_pess_optimistic*)

</div>
<div id="div_fig3">

<img src="tmp_2507.15819/./JV_EuclidIsocurvature1c_pessimisticTickIn.png" alt="Fig11.1" width="50%"/><img src="tmp_2507.15819/./JV_EuclidIsocurvature1d_pessimisticTickIn.png" alt="Fig11.2" width="50%"/>

**Figure 11. -**  Matter power spectra at redshift $z=0$ for Case 2 (left panel) and Case 3 (right panel). The left panel shows the different contributions for our forecasted pessimistic $\GCsp$$1\sigma$ upper bound on our Case 2 uncorrelated isocurvature model with $n_\mathrm{iso}=1.0$ when the underlying true model (fiducial, cyan colour) is purely adiabatic. The upper bound on the isocurvature contribution is so small that the linear pure adiabatic (adi) and total linear (tot lin) matter power spectra are indistinguishable by eye in this figure. The right panel shows the same quantities for the forecasted $\GCsp$$1\sigma$ pessimistic upper bound on our Case 3 one-isocurvature-parameter uncorrelated isocurvature model with $n_\mathrm{iso}=2.9$ when the underlying true model (fiducial) is purely adiabatic. Note how, for Case 3, the isocurvature contribution visibly modifies the spectrum for $k/h \gtrsim 0.1 \mathrm{Mpc}^{-1}$. (*fig:1paramisocurvaturecases*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.15819"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

147  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

20  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
